In [12]:
import re
import pandas as pd

# Define file paths
excel_file_path = 'output_hamrobazaar/raw_data.xlsx'
output_file_path = 'output_hamrobazaar/3.cleaned_data.xlsx'

# Read the Excel file
dfs = pd.read_excel(excel_file_path, sheet_name=None)

# Columns to select
selected_columns = ['PRICE', 'KMRANGE', 'BODYTYPE', 'BRANDNAME','MAKEYEAR','ENGINE','COLOR','TRANSMISSION','FUEL']  # Adjust as necessary

# Function to clean and convert the 'PRICE' column
def clean_price(price):
    return int(str(price).replace('NPR', '').replace(',', '')) if pd.notna(price) else price

# Function to clean and convert 'KMRANGE' column
# def clean_kmrange(kmrange):
#     if pd.isna(kmrange) or (isinstance(kmrange, str) and kmrange.strip() in ['', 'Null']):
#         return None
#     return int(kmrange.replace(',', '')) if isinstance(kmrange, str) else kmrange

def clean_kmrange(kmrange):
    if pd.isna(kmrange) or (isinstance(kmrange, str) and kmrange.strip() in ['', 'Null']):
        return None

    # Remove any non-numeric characters
    if isinstance(kmrange, str):
        # Use regular expression to extract numeric characters only
        kmrange = re.sub(r'[^\d]', '', kmrange)
        print(kmrange);
    
    # Convert to integer if possible
    try:
        kmrange = int(kmrange)
    except (ValueError, TypeError):
        return None
    
    # If the value is less than 100, multiply it by 1000
    if kmrange < 199:
        kmrange *= 1000
    
    return kmrange

automatic_pattern = re.compile(r'(Auto|Automatic|Power\s*Steering|PowerSteering)', flags=re.IGNORECASE)
manual_pattern = re.compile(r'Manual', flags=re.IGNORECASE)
drive_pattern = re.compile(r'(2WD|4WD|4 WD|2 WD)', flags=re.IGNORECASE)
SUV_pattern = re.compile(r'SUV', flags=re.IGNORECASE);
HatchBack_pattern=re.compile(r'Hatchback', flags=re.IGNORECASE);


# Create a writer for the new Excel file
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    
    # Iterate through sheets and process
    for sheet_name, df in dfs.items():
        if sheet_name != 'All Data':
            df[selected_columns].to_excel(writer, sheet_name=sheet_name, index=False)
    
    # Process 'All Data' sheet
    # all_data_sheet = dfs.get('All Data', pd.DataFrame())
    all_data_sheet = next(iter(dfs.values()), pd.DataFrame())

    if not all_data_sheet.empty:
       
        all_data_sheet_copy = all_data_sheet[selected_columns].copy()
        

        # Remove rows with 'PRICE' containing 'NEGOTIABLE'

        # Clean 'PRICE' column
        if 'PRICE' in all_data_sheet_copy.columns:
            all_data_sheet_copy['PRICE'] = all_data_sheet_copy['PRICE'].apply(clean_price)
        # if 'FUEL' in all_data_sheet_copy.columns:
        #     all_data_sheet_copy = all_data_sheet_copy[all_data_sheet_copy['FUEL'].isin(['Petrol', 'Diesel'])]
        # Clean 'KMRANGE' column
        if 'KMRANGE' in all_data_sheet_copy.columns:
            all_data_sheet_copy['MILEAGE'] = all_data_sheet_copy['KMRANGE'].apply(clean_kmrange)
       
        if 'TRANSMISSION' in all_data_sheet_copy.columns:
            all_data_sheet_copy['GEARBOX']  = all_data_sheet_copy['TRANSMISSION'].apply(lambda x: 'Automatic' if pd.notna(x) and automatic_pattern.search(str(x)) else 'Manual' if pd.notna(x) and manual_pattern.search(str(x)) else '-')
            all_data_sheet_copy['DRIVE'] = (all_data_sheet_copy['TRANSMISSION'].apply(lambda x: drive_pattern.search(str(x)) and drive_pattern.search(str(x)).group(0))).replace(" ", "")
        if 'BODYTYPE' in all_data_sheet_copy.columns:
            all_data_sheet_copy['BODY'] = all_data_sheet_copy['BODYTYPE'].apply(lambda x: 'SUV' if pd.notna(x) and SUV_pattern.search(str(x)) else 'Hatchback' if pd.notna(x) and HatchBack_pattern.search(str(x)) else x)

        all_data_sheet_copy = all_data_sheet_copy.drop(columns=['BODYTYPE', 'TRANSMISSION','KMRANGE'], errors='ignore')



        # Write modified 'All Data' to new sheet

        all_data_sheet_copy.to_excel(writer, sheet_name='Sheet1', index=False)

print("Done!")


80000
70000
94000
4000000
38200
35000
45000
30000
36000
28000
44000
35000
80000
600000
50000
55000
58000
50000
58000
43000
2600
10800
45000
46000
36000
100000
76000
36000
80000
87500
80000
38000
124
65000
55000
14000
62000
67800
81000
5000
52000
46000
55000
47000
54000
65000
44000
70000
52321
55343
52
58000
66000
54750
23000
63500
16000
100000
95000
15000
66000
51000
45000
44000
59000
42000
29000
73000
52000
4700
3500
23000
60
71121
55500
30000
5000
39000
66000
67000
124
1400
48000
29000
77000
50000
150000
11000
70000
29050
52000
80000
62000
53000
72000
66000
99000
55000
77000
44500
42500
49000
45100
55100
34100
50000
28000
39000
55400
42000
46100
73
2600
60000
46100
33210
60000
30000
68500
66000
120000
65000
80
18486
65000
17500
65000
26000
51000
49000
50000
41000
19000
59000
78000
48000
25000
45232
60000
80000
0
48000
46000
85000
52000
75
21500
57000
57000
46000
69000
9123
34000
79000
18486
55000
50000
62000
56000
24000
56900
86000
27000
34000
75000
57000
18200
54000
60000
39000
4860

In [4]:
import pandas as pd

working_file_path='M2-1 Working on thesis paper.xlsx'
df = pd.read_excel(working_file_path,sheet_name='All Data')



FileNotFoundError: [Errno 2] No such file or directory: 'M2-1 Working on thesis paper.xlsx'

In [5]:


# Display or use the filtered DataFrame as needed
df.info();


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3406 entries, 0 to 3405
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   1           3406 non-null   int64  
 1   PRICE       3406 non-null   object 
 2   BRANDNAME   3406 non-null   object 
 3   MAKEYEAR    3386 non-null   float64
 4   FUEL        3406 non-null   object 
 5   KM          2811 non-null   object 
 6   BODY        3406 non-null   object 
 7   GEARBOX     3406 non-null   object 
 8   DRIVE       3406 non-null   object 
 9   COLOR       3406 non-null   object 
 10  POSTDATE    3405 non-null   object 
 11  MILEAGE     1676 non-null   object 
 12  ENGINE(CC)  2802 non-null   float64
 13  KM_INT      2394 non-null   float64
 14  PRICE_NEW   3406 non-null   int64  
dtypes: float64(3), int64(2), object(10)
memory usage: 399.3+ KB


https://www.youtube.com/watch?v=Wqmtf9SA_kk&ab_channel=NeuralNine
https://www.youtube.com/watch?v=xu0prYu5e-c&ab_channel=AliNasserEddine